# Week 7: Web scraping, BS4, Selenium and Regexp


  


### Part 1
1. Make a request to: `https://en.wikipedia.org/wiki/Alan_Turing` and print out the responses .text property
2. Find and print the title of the response page
3. Find and print content of the first p tag that has content.
4. Find and print all content from the TOC    
5. Create a dictionary from the TOC links like: {'first link':'#this_is_the_first_link}
6. Create a function that can take a word and look for it in the dictionary keys and then return the content from the first link that matches. Return the next p elements until next headline (h3 element).
7. Make it into a cli program.  

In [5]:
#1 print out the responses .text property
import requests
import bs4

url = "https://en.wikipedia.org/wiki/Alan_Turing"
response = requests.get(url)
response.raise_for_status()
soup = bs4.BeautifulSoup(response.text, 'html.parser')
print(soup.text)






Alan Turing - Wikipedia





































Jump to content





Toggle sidebar












Search















Create accountLog in






Personal tools




 Create account Log in




		Pages for logged out editors learn more


ContributionsTalk













Navigation


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate




Contribute


HelpLearn to editCommunity portalRecent changesUpload file




Tools


What links hereRelated changesUpload fileSpecial pagesPermanent linkPage informationCite this pageWikidata item




Print/export


Download as PDFPrintable version




In other projects


Wikimedia CommonsWikinewsWikiquoteWikisource




Languages

On this Wikipedia the language links are at the top of the page across from the article title. Go to top.















Contents
move to sidebar
hide




(Top)





1Early life and education


				Toggle Early life and education subsection
			




1.1Family







1.2School







1.3Chris

In [4]:
#2 Find and print the title of the response page
from bs4 import BeautifulSoup
import requests

url = "https://en.wikipedia.org/wiki/Alan_Turing"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

#.string-metoden på soup.title returnerer titlen som en streng
title = soup.title.string

print(title)

Alan Turing - Wikipedia


In [5]:
#3 Find and print content of the first p tag that has content.
import textwrap

# soup.select_one() til at finde det første "p" tag, der ikke er tomt
first_p_tag = soup.select_one('p:not(:empty)')

p_text = first_p_tag.get_text()

# Opdel teksten i linjer på højst 80 tegn
lines = textwrap.wrap(p_text, width=80)

# Udskriv hver linje
for line in lines:
    print(line)

#Skal slettes

Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an
English mathematician, computer scientist, logician, cryptanalyst, philosopher,
and theoretical biologist.[6] Turing was highly influential in the development
of theoretical computer science, providing a formalisation of the concepts of
algorithm and computation with the Turing machine, which can be considered a
model of a general-purpose computer.[7][8][9] He is widely considered to be the
father of theoretical computer science and artificial intelligence.[10]


In [173]:
headings = soup.select('div.vector-toc-text')
for heading in headings:
    heading_text = ''
    for child in heading.children:
        if child.name != 'span':
            heading_text += str(child)
    print(heading_text.strip())




(Top)
Early life and education
Family
School
Christopher Morcom
University and work on computability
Career and research
Cryptanalysis
Bombe
Action This Day
Hut 8 and the naval Enigma
Turingery
Delilah
Early computers and the Turing test
Pattern formation and mathematical biology
Personal life
Engagement
Homosexuality and indecency conviction
Treasure
Death
Government apology and pardon
Legacy
Awards, honours, and tributes
Centenary celebrations
Notes and references
Notes
References
Sources
Further reading
Articles
Books
External links


In [7]:
#5 Create a dictionary from the TOC links like: {'first link':'#this_is_the_first_link}

headings = soup.select('div.vector-toc-text')
striped_list = []

for heading in headings:
    heading_text = ''
    for child in heading.children:
        if child.name != 'span':
            heading_text += str(child)
    striped_list.append(heading_text.strip())

striped_list.pop(0)
    
# Opret en dictionary, hvor overskriftsteksten er nøglen og linket er værdien
toc_dict = {}
for key, heading in enumerate(striped_list):
    href = f'#{heading.replace(" ", "_")}'
    toc_dict[heading] = href
    
# Print dictionariet

for key, value in toc_dict.items():
    print(f'{key}: {value}')


Early life and education: #Early_life_and_education
Family: #Family
School: #School
Christopher Morcom: #Christopher_Morcom
University and work on computability: #University_and_work_on_computability
Career and research: #Career_and_research
Cryptanalysis: #Cryptanalysis
Bombe: #Bombe
Action This Day: #Action_This_Day
Hut 8 and the naval Enigma: #Hut_8_and_the_naval_Enigma
Turingery: #Turingery
Delilah: #Delilah
Early computers and the Turing test: #Early_computers_and_the_Turing_test
Pattern formation and mathematical biology: #Pattern_formation_and_mathematical_biology
Personal life: #Personal_life
Engagement: #Engagement
Homosexuality and indecency conviction: #Homosexuality_and_indecency_conviction
Treasure: #Treasure
Death: #Death
Government apology and pardon: #Government_apology_and_pardon
Legacy: #Legacy
Awards, honours, and tributes: #Awards,_honours,_and_tributes
Centenary celebrations: #Centenary_celebrations
Notes and references: #Notes_and_references
Notes: #Notes
Referenc

In [269]:
#6

def search_toc(dictionary, word):
    search_words = word.lower().split()
    for key in dictionary.keys():
        if all(search_word in key.lower() for search_word in search_words):
            link = dictionary[key]
            section = soup.find(id=link[1:])
            content = ''
            for elem in section.find_all_next():
                if elem.name == 'h3':
                    break
                elif elem.name == 'p':
                    content += '\n' + elem.get_text()
            return content.strip()
    return f"No section found matching '{word}'"

word = 'Chr'
result = search_toc(toc_dict, word)
print(f"Content for section related to '{word}':\n{result}")

Content for section related to 'Chr':
At Sherborne, Turing formed a significant friendship with fellow pupil Christopher Collan Morcom (13 July 1911 – 13 February 1930),[36] who has been described as Turing's "first love". Their relationship provided inspiration in Turing's future endeavours, but it was cut short by Morcom's death, in February 1930, from complications of bovine tuberculosis, contracted after drinking infected cow's milk some years previously.[37][38][39]


The event caused Turing great sorrow. He coped with his grief by working that much harder on the topics of science and mathematics that he had shared with Morcom. In a letter to Morcom's mother, Frances Isobel Morcom (née Swan), Turing wrote:
I am sure I could not have found anywhere another companion so brilliant and yet so charming and unconceited. I regarded my interest in my work, and in such things as astronomy (to which he introduced me) as something to be shared with him and I think he felt a little the same a

### Part 2
8. Using regex find out how many times 'Turing' is used in the article
9. Regex: Find all the sentences that has a year in them (sentense defined by: starting at \n or dot or comma and ending at dot or comma.    

In [2]:
#8. Using regex find out how many times 'Turing' is used in the article
from bs4 import BeautifulSoup
import requests
import re

url = "https://en.wikipedia.org/wiki/Alan_Turing"

# Send GET request til URL og gem responsen
response = requests.get(url)

# Parse HTML-indholdet med BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find <body>-tagget og udtræk kun den rene tekst fra tagget
body_text = soup.body.text

# Brug regex til at finde alle forekomster af ordet 'Turing' i teksten
pattern = r'Turing'
matches = re.findall(pattern, body_text)

# Print antallet af forekomster
print(len(matches))



#9 Regex: Find all the sentences that has a year in them (sentense defined by: 
# starting at \n or dot or comma and ending at dot or comma. 

mw_content_text = soup.find('div', {'id': 'mw-content-text'})
p_tags = mw_content_text.find_all('p')
text = '\n'.join(tag.text for tag in p_tags)

pattern2 = r'[\n\.,][^\.]*\b\d{4}\b[^\.]*[\.,]'
matches2 = re.findall(pattern2, text)

# Fjern uønskede linjeskift i de mantch der har et linjeskift
stripped_matches = [match.replace('\n', '') for match in matches2]
print(len(stripped_matches))

"""
#Udskriv de fundne match hvis man vil

count = 1
for i in range(0,len(stripped_matches)):
    print(f' {count}: {stripped_matches[i]}')
    count += 1

"""





438
81


"\n#Udskriv de fundne match hvis man vil\n\ncount = 1\nfor i in range(0,len(stripped_matches)):\n    print(f' {count}: {stripped_matches[i]}')\n    count += 1\n\n"

### Part 3
1. Use selenium to go to amazon.com and search for `Pet Shower Cap - Waterproof Reusable Bath Ear Covers`    
2. Print how many products were found    
3. Find the cheapest and the most expensive product from the list    

In [4]:
# Part3
#1. Use selenium to go to amazon.com and search for `Pet Shower Cap - Waterproof Reusable Bath Ear Covers`

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Opretter en instans af WebDriver (i dette tilfælde Firefox)
driver = webdriver.Firefox()

# Åbner en hjemmeside
driver.get("https://www.google.com")

# Få titlen på siden
title = driver.title

# Udskriv titlen på siden
print(title)

# Søger efter "Selenium" på Google
search_box = driver.find_element_by_name("q")
search_box.send_keys("Selenium")
search_box.send_keys(Keys.RETURN)

# Vent på, at søgeresultaterne vises
driver.implicitly_wait(10)

# Udskriv antallet af søgeresultater
results = driver.find_elements_by_css_selector("h3")
print(len(results))

# Lukker browseren
driver.quit()



WebDriverException: Message: Process unexpectedly closed with status 1
